In [1]:
import json
import gzip
import bz2
import lzma
import glob
from os.path import basename
from  collections import *
from tqdm.notebook import tqdm
import ast

In [5]:
files = glob.glob('/home/kalkiek/projects/reddit-political-affiliation/data/processed/*.tsv')

month_to_files = defaultdict(list)
for fname in files:
    name = basename(fname)
    year_month = name.split('.')[0][3:]
    month_to_files[year_month].append(fname)
print(len(month_to_files))

194


In [20]:
def to_map(flairs):
    items = flairs.split("},")
    m = {}
    for item in items:
        item = item[:-1].replace("{'",'').replace("'}",'')
        sf = item.split("': ")
        if sf[0][0] == ' ':
            sf[0] = sf[0][2:]
        if sf[1][-1] == ' ':
            sf[1] = sf[1][:-2]
        if len(sf[1]) > 0 and sf[1][-1] == "'":
            sf[1] = sf[1][1:-1]            
        #print(sf)
        m[sf[0]] = sf[1]
    return m
#print(to_map("{'bicycling': {'n(5) + 1'}, 'Tribes': {'Nishiki NA West'}, 'bmx': {'Salty WTP'}}"))             
#print(to_map("{'comicbooks': 'Atomic Robo'}"))   
print(to_map("{'MLS': {'New York City FC'}, 'NBA2k': {' '}}"))

{'MLS': 'New York City FC', 'NBA2k': ''}


In [43]:
d = ast.literal_eval("{'MLS': {'New York City FC'}, 'NBA2k': {' '}}")
type(d['MLS'])

set

In [6]:
flair_counts = Counter()

for month, mfiles in tqdm(month_to_files.items(), total=len(month_to_files)):
    user_to_political_affiliations = defaultdict(Counter)
    for fname in mfiles:
        #print(fname)
        with open(fname, 'rt') as f:
            for line in f:
                cols = line[:-1].split('\t')
                user = cols[0]

                try:
                    #sub_flairs = to_map(cols[1])
                    sub_flairs = ast.literal_eval(cols[1])
                    # print(sub_flairs)
                    if not isinstance(subflairs, dict):
                        print('weird line: ' + sub_flairs)
                        continue
                    for k,v in sub_flairs.items():
                        sub_flairs[k] = list(v)[0]
                    print(sub_flairs)
                except:
                    #print(cols[1])
                    pass
                for sub, flair in sub_flairs.items():
                    flair_counts[flair] += 1  

TypeError: unhashable type: 'list'

In [24]:
for s, c in flair_counts.most_common(200):
    print(s, c)

Text 270675
Patriots 241983
Saved by Thanos 235014
 226710
♀ 208836
New User 208407
PC 196019
United States 188937
Lakers 162419
Packers 162417
PS4 161454
Cowboys 160626
♂ 159906
1 154472
Manchester United 152646
Seahawks 145247
Eagles 144626
Arsenal 140687
Liverpool 140513
This user has not yet been verified. 134439
16 125549
{""I don't feel so good""}} 123194
Male 118191
Warriors 118083
Raptors 117340
17 116288
DM 115047
Celtics 113803
Canada 112466
Vikings 109266
Broncos 107400
49ers 107248
Bears 107176
15 105407
\u200b 105363
Bulls 99731
Chelsea 97359
verified 93298
Giants 91052
Steelers 86877
Zerg 85822
Atheist 85435
Cavaliers 84496
Knicks 84120
Spurs 80472
Not a(n) LEO / Unverified User 79205
USA 79116
Lions 78226
New 74517
18 73351
Rockets 72283
14 71417
2 69582
Ravens 69573
Ontario 69024
Panthers 68568
Specs/Imgur Here 67420
Chicago Cubs 66424
76ers 65786
Specs/Imgur here 65315
Protoss 64958
United Kingdom 64944
Browns 64740
TOR - NHL 63478
Heat 63263
new user/low karma 63198
C

In [28]:
with open('/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/flairs-by-freq.tsv', 'wt') as outf:
    for s, c in flair_counts.most_common():
        outf.write('%s\t%d\n' % (s, c))

In [29]:
flair_to_label = {}
with open('/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/flairs-by-label.tsv', 'rt') as f:
    for line in f:
        cols = line[:-1].split(',')
        flair_to_label[cols[0]] = cols[1]
print(flair_to_label)

{'MAGA': 'Republican', 'Democrat': 'Democrat', 'Liberal Democrat': 'Democrat', 'Republican': 'Republican', 'Liberal': 'Democrat', 'Conservative': 'Republican', 'Reagan Conservative': 'Republican', 'Constitutional Conservative': 'Republican', 'Moderate Conservative': 'Republican', 'Fiscal Conservative': 'Republican', 'Bernie Squad - Cadet': 'Democrat'}


In [38]:
output_dir = '/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/'

for month, mfiles in tqdm(month_to_files.items(), total=len(month_to_files)):
    user_to_political_affiliations = defaultdict(Counter)
    for fname in mfiles:
        #print(fname)
        with open(fname, 'rt') as f:
            for line in f:
                cols = line[:-1].split('\t')
                user = cols[0]
                try:
                    sub_flairs = to_map(cols[1])
                except:
                    pass                           
                
                for sub, flair in sub_flairs.items():
                    if flair not in flair_to_label:
                        continue
                    label = flair_to_label[flair]
                    user_to_political_affiliations[user][label] += 1
                    
    print('Saw %d users with political affiliations in %s' % (len(user_to_political_affiliations), month))
    with open(output_dir + month + '.tsv', 'wt') as outf:
        for user, scs in user_to_political_affiliations.items():
            for label, count in scs.items():
                outf.write(user + '\t' + label + '\t' + str(count) + '\n')      

Saw 117 users with political affiliations in 2012-07
Saw 0 users with political affiliations in 2009-04
Saw 0 users with political affiliations in v2_2010-08
Saw 0 users with political affiliations in 2006-04
Saw 0 users with political affiliations in 2008-11
Saw 1639 users with political affiliations in 2018-11
Saw 9 users with political affiliations in 2010-12
Saw 0 users with political affiliations in v2_2010-02
Saw 0 users with political affiliations in 2007-10
Saw 1370 users with political affiliations in 2019-04
Saw 0 users with political affiliations in 2018-05
Saw 0 users with political affiliations in 2008-05
Saw 9 users with political affiliations in 2011-02
Saw 0 users with political affiliations in 2017-09
Saw 0 users with political affiliations in 2009-01
Saw 0 users with political affiliations in 2007-08
Saw 0 users with political affiliations in v2_2010-01
Saw 0 users with political affiliations in 2017-08
Saw 1426 users with political affiliations in 2018-12
Saw 0 users

Saw 182 users with political affiliations in 2014-11
Saw 0 users with political affiliations in v2_2009-06
Saw 12 users with political affiliations in 2011-09
Saw 0 users with political affiliations in 2007-05
Saw 376 users with political affiliations in 2015-12
Saw 0 users with political affiliations in 2017-06
Saw 141 users with political affiliations in 2014-03
Saw 0 users with political affiliations in 2007-06
Saw 8 users with political affiliations in 2010-11
Saw 0 users with political affiliations in v2_2008-10
Saw 0 users with political affiliations in 2006-11
Saw 0 users with political affiliations in v2_2009-12
Saw 0 users with political affiliations in 2009-05
Saw 284 users with political affiliations in 2012-11
Saw 129 users with political affiliations in 2014-01
Saw 8 users with political affiliations in 2011-05
Saw 0 users with political affiliations in 2005-12
Saw 0 users with political affiliations in 2017-10
Saw 0 users with political affiliations in v2_2010-07
Saw 155 